# MSPrior

**MSPrior** is a multi(scale/stream) prior model for realtime temporal learning using RAVE models created by Antoine Caillon. 
\
[Source code on Github](https://github.com/caillonantoine/msprior)
\
\
**RAVE** is a variational autoencoder for fast and high-quality neural audio synthesis by Antoine Caillon and Philippe Esling.
\
[Article on arxiv](https://arxiv.org/abs/2111.05011) & [Source code on Github](https://github.com/acids-ircam/RAVE)

----
<small>*Note that this notebook has been tested with RAVE models until version 2.2.2 - using models created with version >= 2.3. might not lead to a successful training.*
\
\
Last updated: 17.03.2024 \
Author: Martin Heinze

## Setup

*Note: the current version (24.10.2023) of MSPrior training stops at epoch 1000. In order to run the training beyond 1000 epochs, the setup script below downloads a different train.py from devstermarts' fork of MSPrior where the training length can be adjusted to a max amount of epochs. The training cell after the preprocessing step contains command lines for both, the default training as well as the alternative version.*

In [ ]:
#Setup and install miniconda
!curl -L https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh -o miniconda.sh
!chmod +x miniconda.sh
!sh miniconda.sh -b -p /content/miniconda

#Setup and install MSPRIOR
#Pin to version if required
!/content/miniconda/bin/pip install acids-msprior==1.1.2

#Download devstermarts train.py (to train > 1000 epochs)
!mkdir /content/msprior/
!mkdir /content/msprior/msprior_scripts/
!curl -L https://raw.githubusercontent.com/devstermarts/msprior/master/msprior_scripts/train.py -o /content/msprior/msprior_scripts/train.py

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Preprocess

In [ ]:
#Preprocess
!/content/miniconda/bin/msprior preprocess \
--audio /your/audiofiles/ \
--out_path /your/preprocessed/dataset/ \
--rave /your/pretrained/rave/model.ts

#For all possible parameters, check preprocess.py in /msprior_scripts

## Training

In [ ]:
#Start Tensorboard to monitor training progress
%cd /your/runs/

%load_ext tensorboard
%tensorboard --logdir .

In [ ]:
#Train your prior model
%cd /your/runs/

##Use this for original train.py script. Default training length is set to 1000 epochs.
#!/content/miniconda/bin/msprior train \
#--db_path /your/preprocessed/dataset/ \
#--name name-of-your-training \
#--config recurrent # <-- Options are: decoder_only / recurrent / encoder_decoder / encoder_decoder_continuous

#Default: use devstemarts' train.py. Set flag 'epochs' to any integer, use '-1' for continuous training with manual stopping.
!/content/miniconda/bin/python /content/msprior/msprior_scripts/train.py \
--db_path /your/preprocessed/dataset/ \
--name name-of-your-training \
--config recurrent \ # <-- Options are: decoder_only / recurrent / encoder_decoder / encoder_decoder_continuous
--epochs 2000

#To resume training add the following to your train command
#--ckpt /runs/your-training-name/version_X/checkpoints/file.ckpt

#For all possible parameters, check train.py in /msprior_scripts and the .gin file to your chosen configuration in /configs

## Export

In [ ]:
#Model Export
%cd /runs/your-training-name/

!/content/miniconda/bin/msprior export \
--run /runs/your-training-name \
--continuous \ #Use for all but discrete RAVE models
--batch_size==64 #Fixes a current export issue with models trained with recurrent configuration

#For all possible parameters, check export.py in /msprior_scripts